In [5]:
import pm4py
from pm4py.objects.ocel.obj import OCEL
from typing import Optional
import pandas as pd
from typing import Literal, Optional, cast

from pm4py.objects.ocel.obj import OCEL
from pydantic.main import BaseModel

import pandas as pd


ocel = pm4py.read_ocel2_sqlite("../../data/event_logs/ContainerLogistics-v1.sqlite")

/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:75: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/util/dataframe_utils.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [22]:
class RelationCountSummary(BaseModel):
    qualifier: str
    source: str
    target: str
    min_count: int
    max_count: int
    sum: int
    
def summarize_relation_counts(
    relation_table: pd.DataFrame,
    qualifier_col: str,
    source_type_col: str,
    target_type_col: str,
    source_id_col: str,
    target_id_col: str,
    direction: Literal["source", "target"],
) -> list[RelationCountSummary]:
    group_key = source_id_col if direction == "source" else target_id_col

    grouped_relations = (
        relation_table.groupby(
            [group_key, qualifier_col, source_type_col, target_type_col]
        )
        .size()
        .reset_index()
        .rename(columns={0: "count"})
    )
    
    summary = (
        grouped_relations.groupby([qualifier_col, source_type_col, target_type_col])[
            "count"
        ]
        .agg(["min", "max", "sum"])
        .reset_index()
        .rename(columns={"min": "min_count", "max": "max_count"})
    )
    

    summaries = [
        RelationCountSummary(
            qualifier=cast(str, row[qualifier_col]),
            source=cast(str, row[source_type_col]),
            target=cast(str, row[target_type_col]),
            min_count=cast(int, row["min_count"]),
            max_count=cast(int, row["max_count"]),
            sum=cast(int, row["sum"]),
        )
        for _, row in summary.iterrows()
    ]

    return summaries


In [23]:

def summarize_e2o_counts(
    ocel: OCEL, direction: Optional[Literal["events", "objects"]] = "events"
) -> list[RelationCountSummary]:
    return summarize_relation_counts(
        relation_table=ocel.relations,
        direction="source" if direction == "events" else "target",
        qualifier_col=ocel.qualifier,
        source_type_col=ocel.event_activity,
        target_type_col=ocel.object_type_column,
        source_id_col=ocel.event_id_column,
        target_id_col=ocel.object_id_column,
    )



In [24]:
summarize_e2o_counts(ocel ,direction="objects")

[RelationCountSummary(qualifier='CR brought to bay', source='Bring to Loading Bay', target='Container', min_count=1, max_count=1, sum=1960),
 RelationCountSummary(qualifier='CR departed', source='Depart', target='Container', min_count=1, max_count=1, sum=1956),
 RelationCountSummary(qualifier='CR laded', source='Load Truck', target='Container', min_count=1, max_count=6, sum=10551),
 RelationCountSummary(qualifier='CR loaded', source='Load to Vehicle', target='Container', min_count=1, max_count=1, sum=1959),
 RelationCountSummary(qualifier='CR moved', source='Drive to Terminal', target='Container', min_count=1, max_count=1, sum=1989),
 RelationCountSummary(qualifier='CR picked', source='Pick Up Empty Container', target='Container', min_count=1, max_count=1, sum=1995),
 RelationCountSummary(qualifier='CR rescheduled', source='Reschedule Container', target='Container', min_count=1, max_count=1, sum=35),
 RelationCountSummary(qualifier='CR stored', source='Place in Stock', target='Containe

In [18]:

def getO2OWithTypes(ocel):
    o2o_with_types = pd.merge(
        ocel.o2o,
        ocel.objects[[ocel.object_id_column, ocel.object_type_column]],
        how="left",
    )
    o2o_with_types = pd.merge(
        o2o_with_types,
        ocel.objects[[ocel.object_id_column, ocel.object_type_column]],
        how="left",
        left_on=f"{ocel.object_id_column}_2",
        right_on=ocel.object_id_column,
        suffixes=["", "_new"],  # type:ignore
    )
    o2o_with_types = o2o_with_types[
        [
            ocel.object_id_column,
            f"{ocel.object_id_column}_2",
            ocel.qualifier,
            ocel.object_type_column,
            f"{ocel.object_type_column}_new",
        ]
    ]
    rename_map = {
        ocel.object_id_column: "source",
        f"{ocel.object_id_column}_2": "target",
        ocel.qualifier: "qualifier",
        ocel.object_type_column: "source_type",
        f"{ocel.object_type_column}_new": "target_type",
    }
    return o2o_with_types.rename(columns=rename_map)  # type:ignore



def summarize_o2o_counts(
    ocel: OCEL, direction: Optional[Literal["source", "target"]] = "source"
):
    o2o = getO2OWithTypes(ocel)
    return summarize_relation_counts(
        relation_table=o2o,
        qualifier_col="qualifier",
        source_type_col="source_type",
        target_type_col="target_type",
        source_id_col="source",
        target_id_col="target",
        direction="target" if direction == "target" else "source",
    )

     source    qualifier     source_type         target_type  count
0       co1    TD for CO  Customer Order  Transport Document      1
1      co10    TD for CO  Customer Order  Transport Document      1
2     co100    TD for CO  Customer Order  Transport Document      1
3     co101    TD for CO  Customer Order  Transport Document      1
4     co102    TD for CO  Customer Order  Transport Document      1
...     ...          ...             ...                 ...    ...
5361    tr2  TR loads CR           Truck           Container    323
5362    tr3  TR loads CR           Truck           Container    336
5363    tr4  TR loads CR           Truck           Container    334
5364    tr5  TR loads CR           Truck           Container    336
5365    tr6  TR loads CR           Truck           Container    334

[5366 rows x 5 columns]


[RelationCountSummary(qualifier='CR contains HU', source='Container', target='Handling Unit', min_count=1, max_count=6, sum=10552),
 RelationCountSummary(qualifier='CR for TD', source='Container', target='Transport Document', min_count=1, max_count=1, sum=1996),
 RelationCountSummary(qualifier='Ersatz VH for TD', source='Transport Document', target='Vehicle', min_count=1, max_count=2, sum=35),
 RelationCountSummary(qualifier='High-Prio VH for TD', source='Transport Document', target='Vehicle', min_count=1, max_count=1, sum=179),
 RelationCountSummary(qualifier='Regular VH for TD', source='Transport Document', target='Vehicle', min_count=1, max_count=1, sum=565),
 RelationCountSummary(qualifier='TD for CO', source='Customer Order', target='Transport Document', min_count=1, max_count=1, sum=594),
 RelationCountSummary(qualifier='TR loads CR', source='Truck', target='Container', min_count=323, max_count=336, sum=1996)]